In [1]:
''''''
import numpy as np
import scipy as sp
import scipy.signal
from tqdm import tqdm
from matplotlib import pyplot as plt
from cnld.api import *
from cnld import fem, bem, util

%matplotlib qt

geom = define.circular_cmut_1mhz_geometry()
geom.controldomain_nr = 3
# geom.radius = 32e-6
geom.controldomain_ntheta = 1
geom.damping_ratio1 = 0.1
geom.damping_ratio2 = 0.1
# print(geom)

# layout = define.matrix_layout(nx=1, ny=1, pitch_x=100e-6, pitch_y=100e-6)
layout = define.hexagonal_layout(nx=3, ny=3, pitch=74e-6, radius=100e-6)
layout.geometries.append(geom)
layout.controldomains = define.generate_controldomainlist(layout)

grids = grid.generate_grids_from_layout(layout, refn=9)
grids.bem.plot()

In [2]:
dbsolver = solve.DatabaseSolver('circular_cmut_1mhz_hexagonal.db', freq_interp=4, overwrite=True)
dbsolver.layout = layout
dbsolver.grids = grids
dbsolver.freqs = 0, 20e6, 100e3

dbsolver.solve()

Running: 100%|██████████| 200/200 [1:19:40<00:00, 23.90s/it]


In [24]:
fs = 40e6
_, vdc = define.linear_ramp(2e-6, 1 / fs)
vdc *= 30
pulse, _ = util.gausspulse(2e6, 0.25, fs=fs)
pulse *= 30

v = np.concatenate((vdc, pulse + 30))

transmit = define.Transmit()
transmit.waveforms.append(define.Waveform(voltage=v))
transmit.fs = fs

n = 1
vmax = np.max(v)
e_0 = 8.85418782e-12
pmax = e_0 / 2 * 50**2 / (geom.gap / geom.eps_r)**2 * 2
k = pmax / (10e-9)**n / 100
print(k)
lmda = 3 / 2 * k * 0.0

geom.contact_k = k
geom.contact_n = n
geom.contact_z0 = -geom.gap
geom.contact_lmda = lmda
layout.geometries = define.GeometryList([geom])

tsolver = solve.TimeSolver()
tsolver.layout = layout
tsolver.transmit = transmit
tsolver.dbfile = 'single_circular_cmut_5mhz.db'
tsolver.times = 0, 20e-6, 4e-9

# tsolver.recalculate_fir(interp=4)
tsolver.setup()
for i in tqdm(tsolver, total=len(tsolver._solver)):
    pass

plt.plot(np.mean(tsolver.displacement, axis=1))

  0%|          | 0/4999 [00:00<?, ?it/s]13834668468749.998
5000it [00:04, 1016.45it/s]                         


In [27]:
plt.plot(tsolver.displacement)

In [28]:
from scipy import fftpack
import scipy as sp

def qfft(s, n=None, fs=1, axis=-1, full=False):

    ndim = s.ndim
    shape = s.shape
    nsample = shape[axis]

    if n is None:
        n = nsample

    if n > nsample:
        pw = [(0, 0)] * ndim
        pw[axis] = (0, n - nsample)
        s = np.pad(s, pw, mode='constant')
    elif n < nsample:
        s = np.take(s, range(n), axis=axis)

    ft = sp.fftpack.fft(s, axis=axis)
    freqs = sp.fftpack.fftfreq(n, 1 / fs)

    cutoff = (n + 1) // 2

    if full:
        return freqs, ft
    else:
        return freqs[:cutoff], np.take(ft, range(cutoff), axis=axis)


def qesd(s, n=None, fs=1, axis=-1, win=None):
    if win is None:
        win = np.ones(s.shape[axis])

    dims = np.ones(s.ndim, int)
    dims[axis] = -1
    win = win.reshape(dims)

    freqs, ft = qfft(s * win, n=n, fs=fs, full=False, axis=axis)
    return freqs, np.abs(ft)**2 * 2 / fs**2


fs = 1 / (tsolver.time[1] - tsolver.time[0])
x = np.mean(tsolver.displacement, axis=1)
freqs, esd = qesd(x, n=None, fs=fs)

plt.plot(freqs / 1e6, 10 * np.log10(esd / esd.max()))
plt.xlim(0, 50)

(0.0, 50.0)

In [6]:
from cnld import database, fem

freqs, ppfr = database.read_patch_to_patch_freq_resp('circular_cmut_1mhz_hexagonal.db')

fr = np.mean(np.sum(ppfr, axis=0), axis=0)
# fr = np.sum(ppfr, axis=0)[8, :]
plt.plot(freqs / 1e6, 10 * np.log10(np.abs(fr) / np.abs(fr).max()), '.-')
plt.xlim(0, 50)

(0.0, 50.0)

In [29]:
print(geom)

  Geometry
   id: 0
   thickness: 1e-06
   shape: 'circle'
   length_x: None
   length_y: None
   radius: 2.6e-05
   density: 3000
   y_modulus: 160000000000.0
   p_ratio: 0.28
   isol_thickness: 1e-07
   eps_r: 7.5
   gap: 3e-07
   electrode_x: None
   electrode_y: None
   electrode_r: 5e-05
   controldomain_nx: None
   controldomain_ny: None
   controldomain_nr: 3
   controldomain_ntheta: 1
   damping_mode1: 0
   damping_mode2: 5
   damping_freq1: 0
   damping_freq2: 0
   damping_ratio1: 0.1
   damping_ratio2: 0.1
   contact_k: 13834668468749.998
   contact_n: 1
   contact_z0: -3e-07
   contact_lmda: 0.0

